In [1]:
from ase_interface import ANIENS
from ase_interface import ensemblemolecule

import hdnntools as hdt

import numpy as np
import  ase
#from ase.build import molecule
#from ase.neb import NEB
#from ase.calculators.mopac import MOPAC
from ase.md.langevin import Langevin
from ase.md.verlet import VelocityVerlet
from ase.io.trajectory import Trajectory
from ase import units

from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

from ase.optimize.fire import FIRE as QuasiNewton

from ase.md.nvtberendsen import NVTBerendsen
from ase.md import MDLogger

#from ase.neb import NEBtools
from ase.io import read, write
from ase.optimize import BFGS, LBFGS

import time

van Der Waals correction will be unavailable. Please install ased3


In [2]:
dir = '/scratch/Research/gutzwiller_research/gutz_new_data/ML-GQMD-data-NEW/case1/train_models/data-U0.001/testset/'

# Molecule file
molfile = '/nh/nest/u/jsmith/scratch/Research/gutzwiller_research/gutz_new_data/ML-GQMD-data-NEW/case1/test_files/data-U0.001/ts0/sys-000.xyz'

# Dynamics file
xyzfile = dir + 'mdcrd.xyz'

# Trajectory file
trajfile = dir + 'traj.dat'

# Optimized structure out:
optfile = dir + 'optmol.xyz'

ntdir = '/scratch/Research/gutzwiller_research/gutz_new_data/ML-GQMD-data-NEW/case1/train_models/data-U0.001/testsets/testset0'

T = 100.0 # Temperature
dt = 0.1
C = 2.0 # Optimization convergence
steps = 40000

ntdir = '/scratch/Research/gutzwiller_research/gutz_new_data/ML-GQMD-data-NEW/case1/train_models/data-U0.001/'
cns = ntdir + '../../params/rX-2.5R_32-2.0A_a8-8.params'
sae = ntdir + '../../params/sae.dat'
nnf = ntdir + 'train'
Nn = 3

#ntdir = '/nh/nest/u/jsmith/Research/gutzwiller_research/DarwinSets/GutzTrain/train_test_U2/U2_co3/'
#cns = ntdir + 'rX-6.0A_32-4.0A_a8-8.params'
#sae = ntdir + 'sae.dat'
#nnf = ntdir + 'train'
#Nn = 5

In [3]:
# Load molecule
mol = read(molfile)
L = 10.9753
mol.set_cell(([[L, 0, 0],
               [0, L, 0],
               [0, 0, L]]))

mol.set_pbc((True, True, True))

In [4]:
# Set NC
aens = ensemblemolecule(cns, sae, nnf, Nn, 1)

# Set ANI calculator
mol.set_calculator(ANIENS(aens,sdmx=20000000.0))

In [5]:
def erg_exp (Rmag):
    return 100.0*np.exp(-3.53*Rmag)

def frc_exp (Rmag,R):
    F = -3.53*erg_exp(Rmag)*(R/Rmag)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
    return F

mol.calc.set_pairwise(erg_exp, frc_exp)

In [6]:
# Optimize molecule
start_time = time.time()
dyn = QuasiNewton(mol)
dyn.run(fmax=C)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

# Save optimized mol
spc = mol.get_chemical_symbols()
pos = mol.get_positions(wrap=False).reshape(1,len(spc),3)

hdt.writexyzfile(optfile, pos, spc)

      Step     Time          Energy         fmax
FIRE:    0 16:36:47      611.970172      564.3758
FIRE:    1 16:36:47      647.832584      668.4303
FIRE:    2 16:36:48      694.160369      783.0689
FIRE:    3 16:36:48      752.465459      761.6857
FIRE:    4 16:36:48      818.762830      900.0445
FIRE:    5 16:36:48      883.956250     1475.0471
FIRE:    6 16:36:48      867.122068      680.8176
FIRE:    7 16:36:48      917.931316      874.4759
FIRE:    8 16:36:48      904.879821      695.4340
FIRE:    9 16:36:48      920.232811      197.2597
FIRE:   10 16:36:48     1303.180891      891.2339
FIRE:   11 16:36:48     1323.304541      597.1639
FIRE:   12 16:36:48     1366.938492      623.4511
FIRE:   13 16:36:48     1441.746170      665.9568
FIRE:   14 16:36:48     1540.027147      716.6395
FIRE:   15 16:36:48     1652.464610      768.1341
FIRE:   16 16:36:48     1778.451780      785.3850
FIRE:   17 16:36:48     1917.158906      785.4592
FIRE:   18 16:36:48     2067.076503      787.4696
F

FIRE:  163 16:36:52     3421.411421        2.3285
FIRE:  164 16:36:52     3421.421538        2.3115
FIRE:  165 16:36:52     3421.362926        2.2899
FIRE:  166 16:36:52     3421.341300        2.4515
FIRE:  167 16:36:52     3421.449197        2.1568
FIRE:  168 16:36:52     3421.532706        2.3131
FIRE:  169 16:36:53     3421.455539        2.4815
FIRE:  170 16:36:53     3421.577209        2.1740
FIRE:  171 16:36:53     3421.659510        2.3897
FIRE:  172 16:36:53     3421.599728        2.4133
FIRE:  173 16:36:53     3422.128833        7.0860
FIRE:  174 16:36:53     3420.862751       21.5445
FIRE:  175 16:36:53     3421.933837        2.3683
FIRE:  176 16:36:53     3421.927922        2.3485
FIRE:  177 16:36:53     3421.917854        2.3277
FIRE:  178 16:36:53     3421.907477        2.3228
FIRE:  179 16:36:53     3421.900671        2.3035
FIRE:  180 16:36:53     3421.900986        2.2920
FIRE:  181 16:36:53     3421.908749        2.2836
FIRE:  182 16:36:53     3421.922708        2.3148


In [7]:
MaxwellBoltzmannDistribution(mol, 10000.0 * units.kB)

# We want to run MD with constant energy using the VelocityVerlet algorithm.
dyn = VelocityVerlet(mol, dt * units.fs)

In [ ]:
# Open MD output
mdcrd = open(xyzfile,'w')

# Open MD output
traj = open(trajfile,'w')

# Define the printer
def storeenergy(a=mol, d=dyn, b=mdcrd, t=traj):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)

    stddev =  a.calc.stddev

    t.write(str(d.get_number_of_steps()) + ' ' + str(ekin / (1.5 * units.kB)) + ' ' + str(epot) + ' ' + str(ekin) + ' ' + str(epot+ekin) + '\n')
    b.write(str(len(a)) + '\n' + str(ekin / (1.5 * units.kB)) + ' Step: ' + str(d.get_number_of_steps()) + '\n')
    c = a.get_positions(wrap=True)
    for j, i in zip(a, c):
        b.write(str('C') + ' ' + str(i[0]) + ' ' + str(i[1]) + ' ' + str(i[2]) + '\n')

    print('Step: %d Energy per atom: Epot = %.3f  Ekin = %.3f (T=%.3fK)  '
          'Etot = %.6f' ' StdDev = %.3f' % (d.get_number_of_steps(), epot, ekin, ekin / (1.5 * units.kB), epot + ekin, stddev))
dyn.attach(storeenergy, interval=25)

In [ ]:
start_time = time.time()
dyn.run(1000000)
print('[ANI Total time:', time.time() - start_time, 'seconds]')


Step: 25 Energy per atom: Epot = 17.183  Ekin = 1.116 (T=8634.616K)  Etot = 18.299385 StdDev = 818.201
Step: 50 Energy per atom: Epot = 17.155  Ekin = 1.132 (T=8755.206K)  Etot = 18.286362 StdDev = 818.312
Step: 75 Energy per atom: Epot = 17.085  Ekin = 1.189 (T=9195.097K)  Etot = 18.273319 StdDev = 820.306
Step: 100 Energy per atom: Epot = 17.044  Ekin = 1.158 (T=8960.910K)  Etot = 18.202541 StdDev = 820.999
Step: 125 Energy per atom: Epot = 17.063  Ekin = 1.101 (T=8516.395K)  Etot = 18.163652 StdDev = 820.604
Step: 150 Energy per atom: Epot = 17.104  Ekin = 1.092 (T=8445.097K)  Etot = 18.195812 StdDev = 819.915
Step: 175 Energy per atom: Epot = 17.131  Ekin = 1.182 (T=9144.818K)  Etot = 18.313413 StdDev = 819.671
Step: 200 Energy per atom: Epot = 17.147  Ekin = 1.149 (T=8890.578K)  Etot = 18.296606 StdDev = 819.283
Step: 225 Energy per atom: Epot = 17.147  Ekin = 1.159 (T=8966.603K)  Etot = 18.306115 StdDev = 819.138
Step: 250 Energy per atom: Epot = 17.131  Ekin = 1.149 (T=8890.079K

Step: 2000 Energy per atom: Epot = 17.085  Ekin = 1.082 (T=8368.487K)  Etot = 18.166919 StdDev = 820.560
Step: 2025 Energy per atom: Epot = 17.135  Ekin = 1.125 (T=8700.069K)  Etot = 18.259143 StdDev = 819.839
Step: 2050 Energy per atom: Epot = 17.170  Ekin = 1.134 (T=8769.854K)  Etot = 18.303782 StdDev = 819.286
Step: 2075 Energy per atom: Epot = 17.179  Ekin = 1.106 (T=8559.280K)  Etot = 18.285130 StdDev = 819.125
Step: 2100 Energy per atom: Epot = 17.135  Ekin = 1.160 (T=8974.744K)  Etot = 18.294893 StdDev = 819.498
Step: 2125 Energy per atom: Epot = 17.078  Ekin = 1.180 (T=9131.163K)  Etot = 18.257863 StdDev = 820.502
Step: 2150 Energy per atom: Epot = 17.056  Ekin = 1.145 (T=8857.287K)  Etot = 18.201340 StdDev = 821.096
Step: 2175 Energy per atom: Epot = 17.068  Ekin = 1.158 (T=8959.451K)  Etot = 18.225862 StdDev = 820.743
Step: 2200 Energy per atom: Epot = 17.124  Ekin = 1.153 (T=8916.619K)  Etot = 18.276770 StdDev = 819.852
Step: 2225 Energy per atom: Epot = 17.188  Ekin = 1.127

Step: 3975 Energy per atom: Epot = 17.082  Ekin = 1.095 (T=8472.567K)  Etot = 18.177160 StdDev = 820.619
Step: 4000 Energy per atom: Epot = 17.082  Ekin = 1.165 (T=9016.558K)  Etot = 18.247025 StdDev = 819.975
Step: 4025 Energy per atom: Epot = 17.112  Ekin = 1.165 (T=9011.620K)  Etot = 18.277257 StdDev = 819.899
Step: 4050 Energy per atom: Epot = 17.126  Ekin = 1.190 (T=9203.079K)  Etot = 18.315332 StdDev = 819.863
Step: 4075 Energy per atom: Epot = 17.119  Ekin = 1.195 (T=9242.909K)  Etot = 18.313525 StdDev = 819.589
Step: 4100 Energy per atom: Epot = 17.124  Ekin = 1.173 (T=9073.929K)  Etot = 18.296573 StdDev = 819.456
Step: 4125 Energy per atom: Epot = 17.140  Ekin = 1.135 (T=8779.429K)  Etot = 18.274622 StdDev = 819.815
Step: 4150 Energy per atom: Epot = 17.132  Ekin = 1.205 (T=9319.085K)  Etot = 18.336793 StdDev = 819.719
Step: 4175 Energy per atom: Epot = 17.127  Ekin = 1.160 (T=8975.090K)  Etot = 18.287407 StdDev = 820.037
Step: 4200 Energy per atom: Epot = 17.108  Ekin = 1.166